In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui

# Disable PyAutoGUI fail-safe
pyautogui.FAILSAFE = False

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize camera
cap = cv2.VideoCapture(0)

# Initialize cursor position
cursor_x, cursor_y = 0, 0
last_cursor_x, last_cursor_y = 0, 0  # Store last cursor position
smoothing_factor = 0.7  # Adjust this value for more or less smoothing
sensitivity_multiplier = 5.0  # Increased sensitivity for better navigation

# Initialize pause state
is_paused = False

# Function to detect swipe gestures
def detect_gesture(hand_landmarks, handedness):
    # Get the positions of the index, middle, and thumb fingers
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # Calculate the distance between the tips of the fingers
    distance = np.linalg.norm(np.array([index_finger_tip.x, index_finger_tip.y]) - np.array([middle_finger_tip.x, middle_finger_tip.y]))
    pinch_distance = np.linalg.norm(np.array([index_finger_tip.x, index_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y]))

    # Detect scrolling based on left middle finger and thumb coming together
    if handedness == "Left":
        if (np.linalg.norm(np.array([middle_finger_tip.x, middle_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05):
            return "scroll_up"  # Scroll up when middle finger and thumb come together
        elif (np.linalg.norm(np.array([ring_finger_tip.x, ring_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05):
            return "scroll_down"  # Scroll down when ring finger and thumb come together

    # Detect pinch gesture using thumb and index finger
    if pinch_distance < 0.05:  # Pinch detected
        return "pinch"
    
    # Detect zoom out gesture using pinky and thumb
    if handedness == "Left" and (np.linalg.norm(np.array([pinky_tip.x, pinky_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05):
        return "zoom_out"  # Zoom out when pinky and thumb come together

    # Detect zoom in gesture using right pinky and thumb
    if handedness == "Right" and (np.linalg.norm(np.array([pinky_tip.x, pinky_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05):
        return "zoom_in"  # Zoom in when right pinky and thumb come together

    # Detect volume up gesture using right thumb and middle finger
    if handedness == "Right":
        if (np.linalg.norm(np.array([thumb_tip.x, thumb_tip.y]) - np.array([middle_finger_tip.x, middle_finger_tip.y])) < 0.05):
            return "volume_up"  # Volume up when thumb and middle finger come together
        # Detect volume down gesture using right ring finger and thumb
        if (np.linalg.norm(np.array([thumb_tip.x, thumb_tip.y]) - np.array([ring_finger_tip.x, ring_finger_tip.y])) < 0.05):
            return "volume_down"  # Volume down when thumb and ring finger come together
    
    # Detect mute gesture using thumb, middle finger, and ring finger
    if handedness == "Right":
        if (np.linalg.norm(np.array([middle_finger_tip.x, middle_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05 and
            np.linalg.norm(np.array([ring_finger_tip.x, ring_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.05):
            return "mute"  # Mute when thumb, middle finger, and ring finger come together

    # Detect fully fisted hand
    if all([landmark.visibility > 0.5 for landmark in hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP:mp_hands.HandLandmark.PINKY_TIP + 1]]):
        return "fist"

def main():
    global cursor_x, cursor_y, last_cursor_x, last_cursor_y, is_paused
    left_clicking = False
    right_clicking = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and find hands only if not paused
        if not is_paused:
            results = hands.process(rgb_frame)

            # Draw hand landmarks and detect gestures
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:  # Process all detected hands
                    mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Determine if the hand is left or right
                    handedness = results.multi_handedness[results.multi_hand_landmarks.index(hand_landmarks)].classification[0].label
                    
                    # Detect gestures based on the hand
                    gesture = detect_gesture(hand_landmarks, handedness)

                    # Bind mouse cursor movement to the right hand
                    if handedness == "Right":
                        # Calculate the average position of all fingertips
                        fingertips = [
                            hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP],
                            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
                        ]
                        avg_x = sum([finger.x for finger in fingertips]) / len(fingertips)
                        avg_y = sum([finger.y for finger in fingertips]) / len(fingertips)

                        cursor_x = int((avg_x * frame.shape[1]) * sensitivity_multiplier)
                        cursor_y = int((avg_y * frame.shape[0]) * sensitivity_multiplier)

                        # Smoothing for stable movement
                        cursor_x = int(cursor_x * smoothing_factor + last_cursor_x * (1 - smoothing_factor))
                        cursor_y = int(cursor_y * smoothing_factor + last_cursor_y * (1 - smoothing_factor))
                        
                        # Move the mouse cursor to the position of the average fingertip
                        pyautogui.moveTo(cursor_x, cursor_y)

                        # Update last cursor position
                        last_cursor_x, last_cursor_y = cursor_x, cursor_y

                        # Handle right pinch for right click
                        if gesture == "pinch":
                            pyautogui.click(button='right')  # Simulate right click

                    # Center cursor when palm is detected
                    if gesture == "palm":
                        screen_center_x = int(frame.shape[1] / 2)
                        screen_center_y = int(frame.shape[0] / 2)
                        pyautogui.moveTo(screen_center_x, screen_center_y)

                    # Display gesture information on the frame
                    cv2.putText(frame, f"Gesture: {gesture}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 120, 0), 2)

                    # Handle left hand gestures
                    if handedness == "Left":
                        if gesture == "scroll_up":
                            pyautogui.scroll(10)  # Scroll up
                        elif gesture == "scroll_down":
                            pyautogui.scroll(-10)  # Scroll down
                        elif gesture == "pinch":
                            pyautogui.click()  # Simulate left click
                        elif gesture == "zoom_out":
                            pyautogui.hotkey('ctrl', '-')  # Zoom out

                    # Handle right hand gestures
                    if handedness == "Right":
                        if gesture == "zoom_in": 
                            pyautogui.hotkey('ctrl', '+')  # Zoom in
                        elif gesture == "volume_up":
                            pyautogui.hotkey('volumeup')  # Simulate volume up
                        elif gesture == "volume_down":
                            pyautogui.hotkey('volumedown')  # Simulate volume down
                        elif gesture == "mute":
                            pyautogui.hotkey('volumemute')  # Simulate mute

        # Display pause status on frame
        status_text = "PAUSED" if is_paused else "ACTIVE"
        status_color = (0, 0, 255) if is_paused else (0, 255, 0)  # Red when paused, Green when active
        cv2.putText(frame, f"Status: {status_text}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2)

        # Display the frame
        cv2.imshow('Gesture Control', frame)

        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('p'):  # Toggle pause state when 'p' is pressed
            is_paused = not is_paused
            print(f"System {'Paused' if is_paused else 'Resumed'}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
System Resumed
System Paused
